In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Activation, add, Flatten, AveragePooling2D, concatenate
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import tensorflow as tf

In [ ]:
width = 32
height = 32

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
from keras.utils import np_utils

In [ ]:
num_classes = 10
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
val_images, val_labels = x_train[:500], y_train[:500]
train_images, train_labels = x_train[500:], y_train[500:]

inception module 사용을 위해서 따로 설정해주는 것이 좋다. filters값은 여러개로 들어오는 형태이다. 논문의 그림을 보면 이해할 수 있다.

*   feature를 효율적으로 추출하기 위해, 1x1, 3x3, 5x5 convolution 연산을 각각 수행



In [ ]:
def inception(x, filters): 
    pre_layer = x
    
    f1,f2,f3,f4 = filters
    # 1x1
    conv1 = Conv2D(f1, kernel_size=(1,1), padding='same', activation='relu')(pre_layer)
    
    # 1x1, 3x3 conv
    conv2 = Conv2D(f4  , kernel_size=(1,1), padding='same', activation='relu')(pre_layer)
    conv2 = Conv2D(f2, kernel_size=(3,3), padding='same', activation='relu')(conv2)

    # 1x1, 5x5 conv
    conv3 = Conv2D(f4, kernel_size=(1,1), padding='same', activation='relu')(pre_layer)
    conv3 = Conv2D(f3, kernel_size=(5,5), padding='same', activation='relu')(conv3)
    # pooling layer, 1x1 conv
    max_pool = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(pre_layer)
    max_pool = Conv2D(f4, kernel_size=(1,1), padding='same')(max_pool)

    # output = [None(batch_size), w,h,c], c 기준 concatenate (axis = -1 )
    concat = concatenate([conv1, conv2, conv3, max_pool], axis=-1)
    
    return concat
    

논문에서 제공한 표를 보면서 모델을 구현하면 비교적 쉽게 사용할 수 있다.
생각보다 복잡해서 구현하기 쉽지않았다.

In [ ]:
input_shape = x_train[0].shape    
inputs = Input(shape=input_shape)

# conv랑 batch 사이에 max pooling 들어가야 하나, cifar 데이터에선 크기 너무 줄어들어서 빼고 진행할 예정이다.
x = Conv2D(64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(192, kernel_size=(3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x) 

#inception형태는 (input, [1x1, 3x3, 5x5conv, 1x1] )

# inception-3a part
x = inception(x,[64,128,32,32])
# inception-3b part
x = inception(x,[128,192,96,64])
x = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(x)

# inception-4a part
x = inception(x,[192,208,48,64])
aux1 = AveragePooling2D(pool_size=(5,5),strides = 3, padding='valid')(x)
aux1 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(aux1)
aux1 = Flatten()(aux1)
aux1 = Dense(512, activation='relu')(aux1)
aux1 = Dense(10, activation='softmax')(aux1)

# inception-4b part
x = inception(x,[160,224,64,64])
# inception-4c part
x = inception(x,[128,256,64,64])

# inception-4d part
x = inception(x,[112,288,64,64])
aux2 = AveragePooling2D(pool_size=(5,5),strides = 3, padding='valid')(x)
aux2 = Conv2D(128, kernel_size=(1,1), padding='same', activation='relu')(aux2)
aux2 = Flatten()(aux2)
aux2 = Dense(832, activation='relu')(aux2)
aux2 = Dense(10, activation='softmax')(aux2)

# inception-4e part
x = inception(x,[256,320,128,128])


# inception-5a part
x = inception(x,[256,320,128,128])
# inception-5b part
x = inception(x,[384,384,128,128])
x = AveragePooling2D(pool_size=(4,4), padding='valid')(x)
x = Dropout(0.4)(x)
x = Flatten()(x)

outputs = Dense(10, activation='softmax')(x)

In [ ]:
model = Model(inputs=inputs, outputs=[aux1, aux2, outputs])

model.compile(optimizer='adam', loss='categorical_crossentropy', loss_weights=[0.3,0.3,1.0], metrics=['accuracy'])
# loss_weights를 통해서 어떤게 중요한지에 대해서 결정하기 위해서 설정해준다. 


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_58 (Conv2D)             (None, 16, 16, 64)   9472        ['input_2[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 16, 16, 64)  256         ['conv2d_58[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 conv2d_59 (Conv2D)             (None, 16, 16, 192)  110784      ['batch_normalization_2[0][0]

In [ ]:
train_datagenerator = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagenerator = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
history = model.fit(train_datagenerator.flow(train_images,train_labels, batch_size = 32), 
                    validation_data = validation_datagenerator.flow(val_images, val_labels, batch_size = 32),
                    epochs = 10)

Epoch 1/10
1547/1547 [==============================] - 104s 55ms/step - loss: 2.6036 - dense_6_loss: 1.5614 - dense_8_loss: 1.6155 - dense_9_loss: 1.6505 - dense_6_accuracy: 0.4243 - dense_8_accuracy: 0.4009 - dense_9_accuracy: 0.3908 - val_loss: 2.1117 - val_dense_6_loss: 1.2723 - val_dense_8_loss: 1.3384 - val_dense_9_loss: 1.3285 - val_dense_6_accuracy: 0.5500 - val_dense_8_accuracy: 0.5140 - val_dense_9_accuracy: 0.5300
Epoch 2/10
1547/1547 [==============================] - 82s 53ms/step - loss: 1.9531 - dense_6_loss: 1.1713 - dense_8_loss: 1.2068 - dense_9_loss: 1.2397 - dense_6_accuracy: 0.5803 - dense_8_accuracy: 0.5671 - dense_9_accuracy: 0.5565 - val_loss: 1.5589 - val_dense_6_loss: 0.9868 - val_dense_8_loss: 0.9506 - val_dense_9_loss: 0.9777 - val_dense_6_accuracy: 0.6540 - val_dense_8_accuracy: 0.6620 - val_dense_9_accuracy: 0.6600
Epoch 3/10
1547/1547 [==============================] - 83s 53ms/step - loss: 1.6106 - dense_6_loss: 0.9733 - dense_8_loss: 0.9910 - dense_9_lo

### loss, accuracy에 대한 설명을 그래프를 통해서 진행해볼 예정이다.